<a href="https://colab.research.google.com/github/JuliaKoryukova/Neural-Networks/blob/main/06_NLTK_pymorphy3_natasha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color='green' size='7'>**Обработка естественного языка. Предобработка
текста: nltk, pymorphy3, natasha**</font>  

**Этапы по очистке и анализу текста**





**Описание плана работы**

1. Формирование набора текстовых данных из 10 песен на одном языке.
2. Загрузка файла с песнями
3. Проведение лемматизации и стемминга с помощью библиотек pymorphy3, natasha (для русского языка).
5. Приведение всех слов к нижнему регистру.
6. Удаление знаков препинания.
7. Удаление стоп-слов с библиотекой nltk.

# <font color='#11a642' size='6'> Подготовка к заданию
Формирование набора текстовых данных из 10 песен на одном языке. Сохранение в файл .txt.

Данные описаны ниже.

Сайт с которого собирались песни для ДЗ:

https://www.beesona.pro/songs/

Список песен:

ДДТ - Что такое осень

Король и Шут - Кукла Колдуна

Божья коровка - Гранитный камушек

Чиж - Фантом

Алла Пугачева - Позови меня с собой

Иванушки International - Тополиный пух

Любовные истории - Школа

Mr. Credo - Медляк

Земфира - Бесконечность

Анна Герман - Нежность

# <font color='#11a642' size='6'> Импорт и установка библиотек

In [ ]:
# установка дополнительной библиотеки
!pip install pymorphy3

In [ ]:
# установка дополнительной библиотеки
!pip install natasha

In [ ]:
import re # регулярные выражения: предобработка текстов, строить признаки-шаблоны на слова и подстроки текста и многое другое
import string # работа со строками (в тч пунктуация)

import nltk # пакет для символьной и статистической обработки естественного языка
from nltk.tokenize import word_tokenize, sent_tokenize #токенизация текста (слов, предложений)
from nltk.corpus import stopwords # набор стоп-слов из библиотеки

from nltk.tokenize.casual import TweetTokenizer # токенизатор с поддержкой особенностей коротких сообщений Twitter
from nltk.tokenize import MWETokenizer # берет строку, которая уже была разделена на токены,
# и повторно токенизирует ее, объединяя выражения из нескольких слов в отдельные токены, используя словарь MWE

from nltk.stem import PorterStemmer # Метод, который принимает слово в качестве входного аргумента и возвращает слово, уменьшается до его корневой формы
from nltk.stem import WordNetLemmatizer # проставить соответствующий POS-тег для каждого слова при обработке текстов
from nltk.stem import SnowballStemmer

from nltk.util import ngrams # модель n-грамм (последовательность из n элементов, символов строки/слова)

# brown корпус, входящим в состав NLTK.
# Он размечен тегами частей речи
from nltk.corpus import brown

# Корпус UnigramTagger - теггер, который выбирает для каждого слова ту часть речи,
# с которой оно чаще всего встречалось в обучающем корпусе.
from nltk.tag import UnigramTagger

import pymorphy3 # Для русского и украинского языка
from pymorphy3 import MorphAnalyzer
# объект для лемматизации MorphVocab
# извлечение эмбеддингов из текста NewsEmbedding
# объект для синтаксического анализа NewsSyntaxParser
# извлечение имен собственных из текста NamesExtractor
# извлечение тэгов по морфологии NewsMorphTagger
# Segmenter
# создание объекта документа (можно создать из предложения, потом с ним работать и парсить, анализировать) Doc
from natasha import MorphVocab, NewsEmbedding, NewsSyntaxParser, NamesExtractor, NewsMorphTagger, Doc, Segmenter, NewsNERTagger

In [ ]:
morph = pymorphy3.MorphAnalyzer() # Создать экземпляр Морфологического анализатора для русского языка

In [ ]:
# загрузка модулей nltk
#Удаление знаков препинания
nltk.download('punkt')

# Удаление стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# Загрузка сегментатора Natasha
segmenter = Segmenter()
emb = NewsEmbedding()
morph_vocab = MorphVocab()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# <font color='#11a642' size='6'> Загрузка данных
Загрузка подготовленного .txt файл с песнями


In [ ]:
!gdown 16OXg7L2OGem2S6-YxQ37MX9hlxNNWHo2

In [ ]:
song_data = ""
with open("/content/songs_data.txt", encoding='utf-8') as song_file:
  song_data=song_file.read()

In [ ]:
song_data = song_data.replace("\n"," ")
song_data

'ДДТ - Что такое осень  Что такое осень? Это небо. Плачущее небо под ногами. В лужах разлетаются птицы с облаками Осень я давно с тобою не был. В лужах разлетаются птицы с облаками Осень я давно с тобою не был.  Осень в небе, жгут корабли Осень мне бы, прочь от земли Там, где в море тонет печаль Осень темная даль.  Что такое осень? Это камни Верность над чернеющей Невою Осень вновь напомнила душе о самом главном Осень я опять лишен покоя. Осень вновь напомнила душе о самом главном Осень я опять лишен покоя.  Осень в небе, жгут корабли Осень мне бы, прочь от земли Там, где в море тонет печаль Осень темная даль.  Что такое осень? Это ветер Вновь играет рваными цепями Осень доползем ли, долетим ли до ответа Что же будет с родиной и с нами. Осень доползем ли, долетим ли до рассвета Что же будет завтра с нами.  Осень в небе, жгут корабли Осень мне бы, прочь от земли Там, где в море тонет печаль Осень темная даль.  Осень в небе, жгут корабли Осень мне бы, прочь от земли Там, где в море тонет

In [ ]:
# для анализа предложений
song_data_sent = ""
with open("/content/songs_data.txt", encoding='utf-8') as song_file:
  song_data_sent=song_file.read()

song_data_sent = song_data_sent.replace("\n"," ")
song_data_sent

'ДДТ - Что такое осень  Что такое осень? Это небо. Плачущее небо под ногами. В лужах разлетаются птицы с облаками Осень я давно с тобою не был. В лужах разлетаются птицы с облаками Осень я давно с тобою не был.  Осень в небе, жгут корабли Осень мне бы, прочь от земли Там, где в море тонет печаль Осень темная даль.  Что такое осень? Это камни Верность над чернеющей Невою Осень вновь напомнила душе о самом главном Осень я опять лишен покоя. Осень вновь напомнила душе о самом главном Осень я опять лишен покоя.  Осень в небе, жгут корабли Осень мне бы, прочь от земли Там, где в море тонет печаль Осень темная даль.  Что такое осень? Это ветер Вновь играет рваными цепями Осень доползем ли, долетим ли до ответа Что же будет с родиной и с нами. Осень доползем ли, долетим ли до рассвета Что же будет завтра с нами.  Осень в небе, жгут корабли Осень мне бы, прочь от земли Там, где в море тонет печаль Осень темная даль.  Осень в небе, жгут корабли Осень мне бы, прочь от земли Там, где в море тонет

# <font color='#11a642' size='6'> Пропроцессинг текстовых данных

## <font color='#11a642' size='5'> Лемматизация с pymorphy3
Лемматизация с помощью библиотеки pymorphy3 (для русского языка)



In [ ]:
# Функция для лемматизации
def lemmatize_text(text):
    # Разбиваем текст на слова
    words = text.split()
    # Лемматизация каждого слова
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(lemmatized_words)

# Выполняем лемматизацию
lemmatized_text = lemmatize_text(song_data_sent)

# Выводим лемматизированный текст
print(lemmatized_text)

ддт - что такой осень что такой осень? это небо. плачущий небо под ногами. в лужа разлетаться птица с облако осень я давно с ты не был. в лужа разлетаться птица с облако осень я давно с ты не был. осень в небе, жечь корабль осень я бы, прочь от земля там, где в море тонуть печаль осень тёмный даль. что такой осень? это камень верность над чернеть нева осень вновь напомнить душа о сам главный осень я опять лишить покоя. осень вновь напомнить душа о сам главный осень я опять лишить покоя. осень в небе, жечь корабль осень я бы, прочь от земля там, где в море тонуть печаль осень тёмный даль. что такой осень? это ветер вновь играть рваный цепь осень доползти ли, долететь ли до ответ что же быть с родина и с нами. осень доползти ли, долететь ли до рассвет что же быть завтра с нами. осень в небе, жечь корабль осень я бы, прочь от земля там, где в море тонуть печаль осень тёмный даль. осень в небе, жечь корабль осень я бы, прочь от земля там, где в море тонуть печаль осень тёмный даль. таять с

## <font color='#11a642' size='5'> Стемминг с pymorphy3

Стемминг с помощью библиотеки pymorphy3 (для русского языка)

In [ ]:
# Функция для стемминга
def stem_text(text):
    # Разбиваем текст на слова
    words = nltk.word_tokenize(text, language='russian')
    # Стемминг каждого слова
    stemmed_words = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(stemmed_words)

# Выполняем стемминг
stemmed_text = stem_text(song_data_sent)

# Выводим стеммированный текст
print(stemmed_text)

ддт - что такой осень что такой осень ? это небо . плачущий небо под нога . в лужа разлетаться птица с облако осень я давно с ты не быть . в лужа разлетаться птица с облако осень я давно с ты не быть . осень в небо , жечь корабль осень я бы , прочь от земля там , где в море тонуть печаль осень тёмный даль . что такой осень ? это камень верность над чернеть нева осень вновь напомнить душа о сам главный осень я опять лишить покой . осень вновь напомнить душа о сам главный осень я опять лишить покой . осень в небо , жечь корабль осень я бы , прочь от земля там , где в море тонуть печаль осень тёмный даль . что такой осень ? это ветер вновь играть рваный цепь осень доползти ли , долететь ли до ответ что же быть с родина и с мы . осень доползти ли , долететь ли до рассвет что же быть завтра с мы . осень в небо , жечь корабль осень я бы , прочь от земля там , где в море тонуть печаль осень тёмный даль . осень в небо , жечь корабль осень я бы , прочь от земля там , где в море тонуть печаль ос

## <font color='#11a642' size='5'> Лемматизация с natasha
Лемматизация с помощью библиотеки natasha (для русского языка)





In [ ]:
# Инициализация необходимых компонентов
segmenter = Segmenter()                  # Для сегментации текста
emb = NewsEmbedding()                    # Для предобученных эмбеддингов
morph_tagger = NewsMorphTagger(emb)     # Инициализация морфологического теггера

In [ ]:
# Создание документа с текстом
doc_song_data_sent = Doc(song_data_sent)

# Сегментация текста на предложения и токены
doc_song_data_sent.segment(segmenter)

# Применение морфологического анализа
doc_song_data_sent.tag_morph(morph_tagger)

# Собираем все леммы в одну строку, игнорируя None
lemmas = ' '.join([token.lemma if token.lemma else token.text for token in doc_song_data_sent.tokens])

# Вывод общего текста
print("Лемматизированный текст:", lemmas)

Лемматизированный текст: ДДТ - Что такое осень Что такое осень ? Это небо . Плачущее небо под ногами . В лужах разлетаются птицы с облаками Осень я давно с тобою не был . В лужах разлетаются птицы с облаками Осень я давно с тобою не был . Осень в небе , жгут корабли Осень мне бы , прочь от земли Там , где в море тонет печаль Осень темная даль . Что такое осень ? Это камни Верность над чернеющей Невою Осень вновь напомнила душе о самом главном Осень я опять лишен покоя . Осень вновь напомнила душе о самом главном Осень я опять лишен покоя . Осень в небе , жгут корабли Осень мне бы , прочь от земли Там , где в море тонет печаль Осень темная даль . Что такое осень ? Это ветер Вновь играет рваными цепями Осень доползем ли , долетим ли до ответа Что же будет с родиной и с нами . Осень доползем ли , долетим ли до рассвета Что же будет завтра с нами . Осень в небе , жгут корабли Осень мне бы , прочь от земли Там , где в море тонет печаль Осень темная даль . Осень в небе , жгут корабли Осень м

## <font color='#11a642' size='5'> Стемминг с natasha

Стемминг с помощью библиотеки natasha (для русского языка)

In [ ]:
# Стемминг с Natasha
# Функция для стемминга
def stem_text(text):
    # Разбиваем текст на слова
    words = nltk.word_tokenize(text, language='russian')
    # Стемминг каждого слова
    stemmed_words = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(stemmed_words)

# Выполняем стемминг
stemmed_text = stem_text(song_data_sent)

# Выводим стеммированный текст
print(stemmed_text)

ддт - что такой осень что такой осень ? это небо . плачущий небо под нога . в лужа разлетаться птица с облако осень я давно с ты не быть . в лужа разлетаться птица с облако осень я давно с ты не быть . осень в небо , жечь корабль осень я бы , прочь от земля там , где в море тонуть печаль осень тёмный даль . что такой осень ? это камень верность над чернеть нева осень вновь напомнить душа о сам главный осень я опять лишить покой . осень вновь напомнить душа о сам главный осень я опять лишить покой . осень в небо , жечь корабль осень я бы , прочь от земля там , где в море тонуть печаль осень тёмный даль . что такой осень ? это ветер вновь играть рваный цепь осень доползти ли , долететь ли до ответ что же быть с родина и с мы . осень доползти ли , долететь ли до рассвет что же быть завтра с мы . осень в небо , жечь корабль осень я бы , прочь от земля там , где в море тонуть печаль осень тёмный даль . осень в небо , жечь корабль осень я бы , прочь от земля там , где в море тонуть печаль ос

## <font color='#11a642' size='5'> Приведение текста к нижнему регистру

In [ ]:
# Приведение текста к нижнему регистру
text_lower = stemmed_text.lower()
text_lower

"ддт - что такой осень что такой осень ? это небо . плачущий небо под нога . в лужа разлетаться птица с облако осень я давно с ты не быть . в лужа разлетаться птица с облако осень я давно с ты не быть . осень в небо , жечь корабль осень я бы , прочь от земля там , где в море тонуть печаль осень тёмный даль . что такой осень ? это камень верность над чернеть нева осень вновь напомнить душа о сам главный осень я опять лишить покой . осень вновь напомнить душа о сам главный осень я опять лишить покой . осень в небо , жечь корабль осень я бы , прочь от земля там , где в море тонуть печаль осень тёмный даль . что такой осень ? это ветер вновь играть рваный цепь осень доползти ли , долететь ли до ответ что же быть с родина и с мы . осень доползти ли , долететь ли до рассвет что же быть завтра с мы . осень в небо , жечь корабль осень я бы , прочь от земля там , где в море тонуть печаль осень тёмный даль . осень в небо , жечь корабль осень я бы , прочь от земля там , где в море тонуть печаль о

## <font color='#11a642' size='5'> Удаление знаков препинания

In [ ]:
# Удаление знаков препинания
text_punc = re.sub(r'[^\w\s]', '', text_lower)
text_punc

'ддт  что такой осень что такой осень  это небо  плачущий небо под нога  в лужа разлетаться птица с облако осень я давно с ты не быть  в лужа разлетаться птица с облако осень я давно с ты не быть  осень в небо  жечь корабль осень я бы  прочь от земля там  где в море тонуть печаль осень тёмный даль  что такой осень  это камень верность над чернеть нева осень вновь напомнить душа о сам главный осень я опять лишить покой  осень вновь напомнить душа о сам главный осень я опять лишить покой  осень в небо  жечь корабль осень я бы  прочь от земля там  где в море тонуть печаль осень тёмный даль  что такой осень  это ветер вновь играть рваный цепь осень доползти ли  долететь ли до ответ что же быть с родина и с мы  осень доползти ли  долететь ли до рассвет что же быть завтра с мы  осень в небо  жечь корабль осень я бы  прочь от земля там  где в море тонуть печаль осень тёмный даль  осень в небо  жечь корабль осень я бы  прочь от земля там  где в море тонуть печаль осень тёмный даль  таять стая 

## <font color='#11a642' size='5'> Удаление стоп-слов с библиотекой nltk


In [ ]:
# Удаление стоп-слов с nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

filtered_text = remove_stop_words(text_punc)
print("Текст без стоп-слов:")
print(filtered_text)

Текст без стоп-слов:
ддт осень осень это небо плачущий небо нога лужа разлетаться птица облако осень давно лужа разлетаться птица облако осень давно осень небо жечь корабль осень прочь земля море тонуть печаль осень тёмный даль осень это камень верность чернеть нева осень вновь напомнить душа главный осень лишить покой осень вновь напомнить душа главный осень лишить покой осень небо жечь корабль осень прочь земля море тонуть печаль осень тёмный даль осень это ветер вновь играть рваный цепь осень доползти долететь ответ родина осень доползти долететь рассвет завтра осень небо жечь корабль осень прочь земля море тонуть печаль осень тёмный даль осень небо жечь корабль осень прочь земля море тонуть печаль осень тёмный даль таять стая город мгла осень знать сколько рваться листва осень вечно право король шут кукла колдун тёмный мрачный коридор цыпочки вор пробираться дышать спугнуть спать давно всё равно чей комната тайком желать заглянуть увидеть бессонница час ночной менять нелюдимый обли

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# <font color='#11a642' size='6'>Выгрузка очищенных данных в файл
Выгрузка данных после препроцессинга в .txt файл

In [ ]:
# Записываем данные в текстовый файл
with open('song_data_cleaned.txt', 'w', encoding='utf-8') as cleaned_file:
    cleaned_file.write(filtered_text)

print("Данные успешно сохранены в song_data_cleaned.txt")

Данные успешно сохранены в song_data_cleaned.txt


Вывод:

В задании были применены этапы по очистке и анализу текста.

В библиотеках Natasha и Pymorphy3 нет возможности нахождения основы слова (стемминга) напрямую. Но есть в библиотеке NLTK.
